This is the project work

Steps:

0. Import packages
1. Loading the data 
2. Ploting an example picture 
3. Creating the network
4. training the neural network

0. Import packages

In [ ]:
import os
import time
import datetime
from shutil import copyfile
import matplotlib.pyplot as plt
import torch
import xarray as xr
import numpy as np
from skimage import measure
import json

1. Loading the data

-Looks like the data is already splited when we get it so we don't need to.

In [ ]:
BASE_PATH_DATA = './skogsstyrelsen/'

In [ ]:
# Read data + corresponding json info (incl ground truth)
img_paths_train = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_train.npy')))
img_paths_val = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_val.npy')))
img_paths_test = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_test.npy')))
json_content_train = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_train.npy'), allow_pickle=True))
json_content_val = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_val.npy'), allow_pickle=True))
json_content_test = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_test.npy'), allow_pickle=True))

In [ ]:
# Get the training split of the MNIST dataset, the ToTensor makes sure we get the data as tensors, not images
mnist_train = torchvision.datasets.MNIST('./', train=True, download=True, transform=torchvision.transforms.ToTensor())

mnist_test = torchvision.datasets.MNIST('./', train=False, download=True, transform=torchvision.transforms.ToTensor())
# Create a DataLoader from the dataset that we'll use to get batches of data during training
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=1000, shuffle=False)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=1000, shuffle=False)

2. Ploting an example picture

In [ ]:
# Function that takes an MNIST tensor and shows the image
def plot_digit(data):
    # Transfrom the images into an appropriate shape for displaying
    data = data.view(28,28)
    plt.imshow(data, cmap='gray')
    plt.show()

# Get the first batch of images and labels from the DataLoader
images, labels = next(iter(test_loader))

# Plot the first image of the batch
plot_digit(images[0])

3. Creating the network

-"CrossEntropyLoss expects class indices and does not take one-hot encoded tensors as target labels." pytorch forums

In [ ]:
#-----------------------------------Network--------------------------------------

epochs=300
network=nn.Sequential(
    nn.Linear(784, 80), # First layer of the network takes the entire image and reduces it to 100 dimensions
    nn.ReLU(),
    nn.Linear(80, 10) # The second layer takes those 100 dimensions and reduces them into estimeated values for each digit
)

loss_function=torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=1)

4. training the neural network

In [ ]:
# Decide the number of epochs to train for (one epoch is one optimization iteration on the entire dataset)
epochs = 10
losses=[]
# For each epoch
for epoch in range(epochs):

    # For each batch of data (since the dataset is too large to run all data through the network at once)
    for (images, labels) in train_loader:

        # Extract the labels and turn them into one-hot representation (note: not all loss functions needs this)

        # Reshape the images to a single vector (28*28 = 784)
        images = images.view(-1,784)

        # Predict for each vector what digit they represent
        prediction = network(data)

        # Calculate the loss of the prediction by comparing to the expected output
        loss = loss_function(prediction, labels)

        # Backpropogate the loss through the network to find the gradients of all parameters
        loss.backward()

        # Update the parameters along their gradients
        optimizer.step()

        # Clear stored gradient values
        optimizer.zero_grad()

        # Add the loss to the total epoch loss (item() turns a PyTorch scalar into a normal Python datatype)
        losses.append(loss.item())
